In [1]:
import pandas as pd
import numpy as np
import wrds
import statsmodels.api as sm
import datetime

from dateutil.relativedelta import relativedelta

# Download CRSP stock information
conn = wrds.Connection()
crsp = conn.raw_sql("""
                      select a.permno, 
                      a.permco,
                      a.date, 
                      b.shrcd, 
                      b.exchcd, 
                      a.ret, 
                      a.vol, 
                      a.shrout, 
                      a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1965' and '12/31/1989'
                      and b.exchcd between 1 and 3
                      and b.shrcd between 10 and 11
                      """)

Enter your WRDS username [rory]:rorysrose
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
df=crsp
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')
df['prc']=df['prc'].apply(lambda x: abs(x))
df['year-month']=df['date'].apply(lambda x:str(x.year)+'-'+str(x.month))
df.sort_values(by=['permno', 'date'], ascending=True, inplace=True)
df=df.reset_index(drop=True)
df['logret'] = np.log(1+df['ret'])

In [6]:
lagged=6 # J-month lagged returns
holdingperiod=6 # K-month holding returns
skipmonth=1

df['momentum_{}_logreturn'.format(lagged)] =\
np.exp(df.groupby('permno')['logret'].rolling(lagged).sum().reset_index(drop=True))-1
df['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)] =\
np.exp(df.groupby('permno')['logret'].rolling(
    holdingperiod).sum().reset_index(drop=True).shift(
    -holdingperiod-skipmonth))-1
df_=df.dropna().reset_index(drop=True)

In [7]:
# remove some extreme values
percentile=0.01
bottom_extreme_value=df_['momentum_{}_logreturn'.format(lagged)].quantile(percentile)
top_extreme_value=df_['momentum_{}_logreturn'.format(lagged)].quantile(1-percentile)
df_=\
df_[(df_['momentum_{}_logreturn'.format(lagged)] >= bottom_extreme_value)&\
    (df_['momentum_{}_logreturn'.format(lagged)] <= top_extreme_value)].reset_index(drop=True)

num_percentile=10
df_['momentum_{}_logreturn_rank_{}_{}'.format(lagged,\
                                              1,\
                                              num_percentile)]=\
df_.groupby('date')['momentum_{}_logreturn'.format(lagged)].transform(
    lambda x: pd.qcut(x.rank(),
                      num_percentile,
                      labels=False))+1

### Fama-MacBeth Regression

In [177]:
result_list = []
f=['momentum_{}_logreturn'.format(lagged)]
time=df_[df_['date'] >= datetime.datetime(1970,6,1)]['date'].sort_values().unique()

result_list = []
beta_list=[]
gamma_list=[]
t_list=[]
for t in time[0:]:
    
    print(t)
    equity=df_[(df_['date'] == t)]['permno'].sort_values().unique()
    df_t=df_[(df_['date'] <= t)&\
             (df_['date'] >= t-relativedelta(years=5))&\
             (df_["permno"].isin(equity))]
    
    beta_t=[]
    for j in equity:
        
#         print("stock={}".format(j))
        df_e = df_t[(df_t['permno'] == j)]
        
        Y = df_e['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)].values/6
        X = sm.add_constant(df_e['momentum_{}_logreturn'.format(lagged)].values)
        reg = sm.OLS(Y,X).fit(cov_type='HAC', cov_kwds={'maxlags': 6}, use_t=True)
        beta=reg.params[0]
        beta_t.append(beta)
    
    ret_t=df_t[(df_t['date'] == t)]['holding_{}_skip_{}_logreturn'.format(holdingperiod, skipmonth)]
    beta_list.append(beta_t)
    
    YY = ret_t.values
    XX = sm.add_constant(beta_t)
    reg = sm.OLS(YY,XX).fit(cov_type='HAC', cov_kwds={'maxlags': 6}, use_t=True)
    
    gamma_list.append(reg.params)
    t_list.append(reg.tvalues)
    

1970-06-30 00:00:00
1970-07-31 00:00:00
1970-08-31 00:00:00
1970-09-30 00:00:00
1970-10-30 00:00:00
1970-11-30 00:00:00
1970-12-31 00:00:00
1971-01-29 00:00:00
1971-02-26 00:00:00
1971-03-31 00:00:00
1971-04-30 00:00:00
1971-05-28 00:00:00
1971-06-30 00:00:00
1971-07-30 00:00:00
1971-08-31 00:00:00
1971-09-30 00:00:00
1971-10-29 00:00:00
1971-11-30 00:00:00
1971-12-31 00:00:00
1972-01-31 00:00:00
1972-02-29 00:00:00
1972-03-30 00:00:00
1972-04-28 00:00:00
1972-05-31 00:00:00
1972-06-30 00:00:00
1972-07-31 00:00:00
1972-08-31 00:00:00
1972-09-29 00:00:00
1972-10-31 00:00:00
1972-11-30 00:00:00
1972-12-29 00:00:00
1973-01-31 00:00:00
1973-02-28 00:00:00
1973-03-30 00:00:00
1973-04-30 00:00:00
1973-05-31 00:00:00
1973-06-29 00:00:00
1973-07-31 00:00:00
1973-08-31 00:00:00
1973-09-28 00:00:00
1973-10-31 00:00:00
1973-11-30 00:00:00
1973-12-31 00:00:00
1974-01-31 00:00:00
1974-02-28 00:00:00
1974-03-29 00:00:00
1974-04-30 00:00:00
1974-05-31 00:00:00
1974-06-28 00:00:00
1974-07-31 00:00:00


### Fama-MacBeth Univariate

#### gamma_0 (constant)

In [192]:
sm.OLS(np.array(gamma_list).T[0],
       [1 for i in range(np.array(gamma_list).T[0].shape[0])]).fit(cov_type='HAC',
                                                                   cov_kwds={'maxlags':lagged},
                                                                   use_t=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 11 Mar 2024   Prob (F-statistic):                nan
Time:                        18:49:44   Log-Likelihood:                 82.734
No. Observations:                 235   AIC:                            -163.5
Df Residuals:                     234   BIC:                            -160.0
Df Model:                           0                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0676      0.022      3.071      0.002       0.024       0.111
==============================================================================
Omnibus:                       17.519   Durbin-Watson:                   0.323
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.547
Skew:                           0.586   Prob(JB):                     3.45e-05
Kurtosis:                       3.850   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 6 lags and without small sample correction
"""

In [199]:
print("estimated coefficient for constant in 2-step Fama-Macbeth: {}, t-stat: {}".format(0.0676, 3.071))

estimated coefficient for constant in 2-step Fama-Macbeth: 0.0676, t-stat: 3.071


#### gamma_1 (Momentum factor)

In [193]:
sm.OLS(np.array(gamma_list).T[1],
       [1 for i in range(np.array(gamma_list).T[1].shape[0])]).fit(cov_type='HAC',
                                                                   cov_kwds={'maxlags':lagged},
                                                                   use_t=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 11 Mar 2024   Prob (F-statistic):                nan
Time:                        18:49:56   Log-Likelihood:                -337.79
No. Observations:                 235   AIC:                             677.6
Df Residuals:                     234   BIC:                             681.0
Df Model:                           0                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6566      0.111      5.891      0.000       0.437       0.876
==============================================================================
Omnibus:                      155.228   Durbin-Watson:                   1.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1176.586
Skew:                           2.626   Prob(JB):                    3.22e-256
Kurtosis:                      12.622   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 6 lags and without small sample correction
"""

In [198]:
print("estimated coefficient for momentum in 2-step Fama-Macbeth: {}, t-stat: {}".format(0.6566, 5.891))

estimated coefficient for momentum in 2-step Fama-Macbeth: 0.6566, t-stat: 5.891
